## **Resumen de textos con Text Rank**

In [181]:
# Se importa las librerias necesarias
import pandas as pd
import numpy as np
import wrap, textwrap
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# nltk sirve para descargar los stopwords y lematizar y tokenizar
# sklearn sirve para vectorizar los textos y calcular la similitud coseno

In [182]:
# Descargar stopwords y lematizar y tokenizar
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\insit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\insit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [183]:
# Se carga el dataset
df = pd.read_csv('news.csv')

# Se muestra el dataset
df.head()

,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra


In [184]:
# Se escoge una noticia aleatoria
news_sample_test = df['news'].sample()

# Se muestra la noticia
print(textwrap.fill(news_sample_test.iloc[0], width=140))

La inflación de Corea del Sur volvió a superar las previsiones acelerándose más allá del 5  por primera vez en más de 13 años lo que llevó
al banco central a señalar nuevas subidas de tipos de interés y al gobierno a intensificar las medidas de alivio de precios.Los precios al
consumidor avanzaron un 54 en mayo con respecto al año anterior más rápido que el pronóstico de los economistas de un 51 y alcanzando el
nivel más alto desde agosto de 2008 informó el viernes la oficina de estadísticas. La lectura de abril de 48 ya fue la más fuerte desde
octubre de 2008.La lectura es el ejemplo más reciente de un crecimiento de los precios que supera las expectativas mientras los responsables
políticos mundiales luchan por controlar la inflación que alguna vez consideraron un fenómeno de corta duración.El Banco de Corea dijo que
la inflación en el rango del 5 probablemente continuaría en junio y julio en un comunicado publicado después de los datos. Es más importante
que cualquier otra cosa gest

In [185]:
# Guardo el texto en una variable
news_sample = textwrap.fill(news_sample_test.iloc[0], width=140)
print(news_sample)

La inflación de Corea del Sur volvió a superar las previsiones acelerándose más allá del 5  por primera vez en más de 13 años lo que llevó
al banco central a señalar nuevas subidas de tipos de interés y al gobierno a intensificar las medidas de alivio de precios.Los precios al
consumidor avanzaron un 54 en mayo con respecto al año anterior más rápido que el pronóstico de los economistas de un 51 y alcanzando el
nivel más alto desde agosto de 2008 informó el viernes la oficina de estadísticas. La lectura de abril de 48 ya fue la más fuerte desde
octubre de 2008.La lectura es el ejemplo más reciente de un crecimiento de los precios que supera las expectativas mientras los responsables
políticos mundiales luchan por controlar la inflación que alguna vez consideraron un fenómeno de corta duración.El Banco de Corea dijo que
la inflación en el rango del 5 probablemente continuaría en junio y julio en un comunicado publicado después de los datos. Es más importante
que cualquier otra cosa gest

In [186]:
# Se separa el texto en oraciones
lineas = news_sample.split('.')
print(lineas)
lineas[0]

['La inflación de Corea del Sur volvió a superar las previsiones acelerándose más allá del 5  por primera vez en más de 13 años lo que llevó\nal banco central a señalar nuevas subidas de tipos de interés y al gobierno a intensificar las medidas de alivio de precios', 'Los precios al\nconsumidor avanzaron un 54 en mayo con respecto al año anterior más rápido que el pronóstico de los economistas de un 51 y alcanzando el\nnivel más alto desde agosto de 2008 informó el viernes la oficina de estadísticas', ' La lectura de abril de 48 ya fue la más fuerte desde\noctubre de 2008', 'La lectura es el ejemplo más reciente de un crecimiento de los precios que supera las expectativas mientras los responsables\npolíticos mundiales luchan por controlar la inflación que alguna vez consideraron un fenómeno de corta duración', 'El Banco de Corea dijo que\nla inflación en el rango del 5 probablemente continuaría en junio y julio en un comunicado publicado después de los datos', ' Es más importante\nque 

'La inflación de Corea del Sur volvió a superar las previsiones acelerándose más allá del 5  por primera vez en más de 13 años lo que llevó\nal banco central a señalar nuevas subidas de tipos de interés y al gobierno a intensificar las medidas de alivio de precios'

In [187]:
# Limpiar el texto y eliminar elementos vacios
lineas = [item for item in lineas if item.strip()]
print(lineas)
len(lineas)

['La inflación de Corea del Sur volvió a superar las previsiones acelerándose más allá del 5  por primera vez en más de 13 años lo que llevó\nal banco central a señalar nuevas subidas de tipos de interés y al gobierno a intensificar las medidas de alivio de precios', 'Los precios al\nconsumidor avanzaron un 54 en mayo con respecto al año anterior más rápido que el pronóstico de los economistas de un 51 y alcanzando el\nnivel más alto desde agosto de 2008 informó el viernes la oficina de estadísticas', ' La lectura de abril de 48 ya fue la más fuerte desde\noctubre de 2008', 'La lectura es el ejemplo más reciente de un crecimiento de los precios que supera las expectativas mientras los responsables\npolíticos mundiales luchan por controlar la inflación que alguna vez consideraron un fenómeno de corta duración', 'El Banco de Corea dijo que\nla inflación en el rango del 5 probablemente continuaría en junio y julio en un comunicado publicado después de los datos', ' Es más importante\nque 

24

In [188]:
# Se vectoriza el texto
featurizer = TfidfVectorizer(
    stop_words=stopwords.words('spanish'),
    norm='l1',
)

In [189]:
# Se crea la matriz X
X = featurizer.fit_transform(lineas)
X

<24x234 sparse matrix of type '<class 'numpy.float64'>'
	with 311 stored elements in Compressed Sparse Row format>

In [190]:
# Se calcula la matriz de similitud
sim_matrix = cosine_similarity(X)
sim_matrix.shape


(24, 24)

In [191]:
# Se muestra la matriz de similitud
sim_matrix

# La matriz de similitud es una matriz cuadrada de tamaño 10x10
# La diagonal de la matriz es 1, ya que la similitud de una oracion consigo misma es 1
# La matriz es simetrica, ya que la similitud de la oracion i con la oracion j es la misma que la similitud de la oracion j con la oracion i
# La matriz es dispersa, ya que la mayoria de los valores son 0
# La matriz es densa, ya que la mayoria de los valores son 0


array([[1.        , 0.01500799, 0.        , 0.06625029, 0.08251483,
        0.04308101, 0.01998889, 0.11202355, 0.04442911, 0.03350764,
        0.        , 0.        , 0.03177163, 0.        , 0.        ,
        0.06027648, 0.        , 0.05508519, 0.15261495, 0.04808296,
        0.        , 0.        , 0.03641943, 0.01604483],
       [0.01500799, 1.        , 0.07953019, 0.01673598, 0.        ,
        0.0221367 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.04401536,
        0.01631579, 0.        , 0.        , 0.06966541, 0.27343597,
        0.15641742, 0.        , 0.03954182, 0.01742042],
       [0.        , 0.07953019, 1.        , 0.08168401, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.07898417,
        0.        , 0.        , 0.        , 0.        ],
       [0.066

In [192]:
# Se normaliza la matriz de similitud
sim_matrix = sim_matrix / sim_matrix.sum(axis=1, keepdims=True)

# Se muestra la matriz de similitud
(sim_matrix)

array([[0.55032782, 0.00825931, 0.        , 0.03645938, 0.0454102 ,
        0.02370868, 0.01100044, 0.06164967, 0.02445058, 0.01844019,
        0.        , 0.        , 0.01748481, 0.        , 0.        ,
        0.03317182, 0.        , 0.03031491, 0.08398825, 0.02646139,
        0.        , 0.        , 0.02004262, 0.00882991],
       [0.0085749 , 0.57135575, 0.04544003, 0.0095622 , 0.        ,
        0.01264793, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.02514843,
        0.00932212, 0.        , 0.        , 0.03980373, 0.15622921,
        0.08936999, 0.        , 0.02259245, 0.00995326],
       [0.        , 0.06412699, 0.80632262, 0.06586367, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.06368673,
        0.        , 0.        , 0.        , 0.        ],
       [0.038

In [193]:
sim_matrix[19].sum()

0.9999999999999999

In [194]:
# Suavizado y regularizacion
# Matriz de transicion uniforme
U = np.ones_like(sim_matrix) / len(sim_matrix)
U

array([[0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667],
       [0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667],
       [0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667],
       [0.041

In [195]:
# Matriz de similitud suavizada
factor = 0.15
sim_matrix = (1 - factor) * U + factor * sim_matrix
sim_matrix

array([[0.11796584, 0.03665556, 0.03541667, 0.04088557, 0.0422282 ,
        0.03897297, 0.03706673, 0.04466412, 0.03908425, 0.03818269,
        0.03541667, 0.03541667, 0.03803939, 0.03541667, 0.03541667,
        0.04039244, 0.03541667, 0.0399639 , 0.0480149 , 0.03938588,
        0.03541667, 0.03541667, 0.03842306, 0.03674115],
       [0.0367029 , 0.12112003, 0.04223267, 0.036851  , 0.03541667,
        0.03731386, 0.03541667, 0.03541667, 0.03541667, 0.03541667,
        0.03541667, 0.03541667, 0.03541667, 0.03541667, 0.03918893,
        0.03681498, 0.03541667, 0.03541667, 0.04138723, 0.05885105,
        0.04882217, 0.03541667, 0.03880553, 0.03690966],
       [0.03541667, 0.04503571, 0.15636506, 0.04529622, 0.03541667,
        0.03541667, 0.03541667, 0.03541667, 0.03541667, 0.03541667,
        0.03541667, 0.03541667, 0.03541667, 0.03541667, 0.03541667,
        0.03541667, 0.03541667, 0.03541667, 0.03541667, 0.04496968,
        0.03541667, 0.03541667, 0.03541667, 0.03541667],
       [0.041

In [196]:
sim_matrix.shape

(24, 24)

In [197]:
# Calcular los eigenvectores y eigenvalores
eigenvalues, eigenvectors = np.linalg.eig(sim_matrix.T)

In [198]:
# Se imprimen los eigenvectores y eigenvalores
eigenvalues


array([1.        , 0.15      , 0.14189004, 0.13363533, 0.05318189,
       0.12657105, 0.12214563, 0.06115474, 0.06308086, 0.06416916,
       0.11541479, 0.11161459, 0.07100558, 0.07479815, 0.075609  ,
       0.10574227, 0.10441578, 0.09927901, 0.09727543, 0.08158999,
       0.0842945 , 0.09125688, 0.08949642, 0.08759136])

In [199]:

eigenvectors[:,0]


array([-0.20496147, -0.20445794, -0.20194231, -0.20475192, -0.20634896,
       -0.20374471, -0.20257011, -0.20645971, -0.20305873, -0.20195744,
       -0.20375016, -0.20364246, -0.20312168, -0.20411779, -0.20352038,
       -0.20524394, -0.20324754, -0.20308608, -0.20656699, -0.20891316,
       -0.2030528 , -0.20383372, -0.20394968, -0.20252718])

In [200]:
# Se ordenan los eigenvectores y eigenvalores
scores = eigenvectors[:,0]
sort_idx = np.argsort(-scores)
sort_idx


array([ 2,  9, 23,  6, 20,  8, 17, 12, 16, 14, 11,  5, 10, 21, 22, 13,  1,
        3,  0, 15,  4,  7, 18, 19], dtype=int64)

In [201]:
# Se imprime el resumen
print('Crear un resumen de la noticia:')
for i in sort_idx[:5]:
    print(textwrap.fill('%.2f: %s' % (scores[i], lineas[i])))

Crear un resumen de la noticia:
-0.20:  La lectura de abril de 48 ya fue la más fuerte desde octubre
de 2008
-0.20:  Calificó el resultado como en gran medida dentro del rango
esperado por el banco
-0.20:  El precio de los enseres y servicios del hogar subió un 54Los
precios en hoteles y restaurantes subieron un 73 en medio de las
restricciones más relajadas de Covid en la actividad pública
-0.20: El gobernador del BOK Rhee Changyong ya advirtió la semana
pasada que esperaba que la inflación superara el 5 y se mantuviera
allí durante algún tiempo
-0.20: Los costos de transporte lideraron las ganancias aumentando un
145 respecto al año anterior
